<a href="https://colab.research.google.com/github/RafsanJany-44/Machine-School/blob/main/Particle_Swarm_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [1]:
import numpy as np
import pandas as pd
dataset = pd.read_csv("/content/Fitbit_feature_merged_version_2023.06.26_ST-II&ST-V_Pre.csv")
target = "Activity"
result = {}

In [2]:
dataset.dropna(axis=0, how='any', inplace = True)
dataset = dataset.reset_index(drop=True)

In [3]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

In [4]:

row = 300
X = dataset.iloc[:row,11:]
y = dataset.loc[:row-1,target]

In [5]:
X.shape

(300, 73)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
"""X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values"""

'X_train = X_train.values\nX_test = X_test.values\ny_train = y_train.values\ny_test = y_test.values'

In [8]:
!pip install niapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 14.2 MB/s eta 0:00:00


#Feature Selections

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.kernel_ridge import KernelRidge

from niapy.problems import Problem
from niapy.task import Task
from niapy.algorithms.basic import ParticleSwarmOptimization



def Swarm_Optzr(obj,X_main,X_trn,y_trn,least_score = 0.5):
  if "<class 'pandas.core.frame.DataFrame'>" == str(type(X_train)):
    X_trn = X_trn.values
  if "<class 'pandas.core.frame.DataFrame'>" == str(type(X_train)):
    y_trn = y_trn.values

  problem = obj(X_trn, y_trn)
  task = Task(problem, max_iters=100)
  algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
  best_features, best_fitness = algorithm.run(task)

  selected_features = best_features > least_score
  #print('Number of selected features:', selected_features.sum())
  #print('Selected features:', ', '.join(X.columns[selected_features].tolist()))
  return X_main.columns[selected_features].tolist()





class RidgeFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(KernelRidge(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)





class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)



class NBFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(GaussianNB(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)



class KNNFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(KNeighborsClassifier(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)



class RFFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(RandomForestClassifier(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)



class HGBCFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(HistGradientBoostingClassifier(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)




In [11]:
X.columns

Index(['nni_counter', 'nni_mean', 'nni_min', 'nni_max', 'hr_mean', 'hr_min',
       'hr_max', 'hr_std', 'nni_diff_mean', 'nni_diff_min', 'nni_diff_max',
       'sdnn', 'rmssd', 'sdsd', 'nn50', 'pnn50', 'nn20', 'pnn20', 'tinn_n',
       'tinn_m', 'tinn', 'tri_index', 'fft_peak_low', 'fft_peak_medium',
       'fft_peak_high', 'fft_abs_low', 'fft_abs_medium', 'fft_abs_high',
       'fft_rel_low', 'fft_rel_medium', 'fft_log_low', 'fft_log_medium',
       'fft_log_high', 'fft_norm_low', 'fft_norm_high', 'fft_ratio',
       'fft_total', 'lomb_peak_low', 'lomb_peak_medium', 'lomb_peak_high',
       'lomb_abs_low', 'lomb_abs_medium', 'lomb_abs_high', 'lomb_rel_low',
       'lomb_rel_medium', 'lomb_rel_high', 'lomb_log_low', 'lomb_log_medium',
       'lomb_log_high', 'lomb_norm_high', 'lomb_norm_Low', 'lomb_ratio',
       'lomb_total', 'ar_abs_low', 'ar_abs_medium', 'ar_abs_high',
       'ar_rel_low', 'ar_rel_medium', 'ar_rel_high', 'ar_log_low',
       'ar_log_medium', 'ar_log_high', 'ar_norm_

In [48]:
print(Swarm_Optzr(RidgeFeatureSelection,X,X_train,y_train))

['nni_counter', 'nni_min', 'hr_max', 'nni_diff_min', 'rmssd', 'nn20', 'pnn20', 'tinn_n', 'fft_peak_low', 'fft_abs_low', 'fft_log_high', 'fft_ratio', 'lomb_peak_medium', 'lomb_abs_low', 'lomb_abs_high', 'lomb_rel_low', 'lomb_rel_high', 'lomb_log_low', 'lomb_norm_high', 'lomb_ratio', 'lomb_total', 'ar_abs_medium', 'ar_abs_high', 'ar_rel_low', 'ar_log_low', 'ar_log_medium', 'sd1', 'sd2', 'sd_ratio', 'sampen']


In [ ]:
rid = Swarm_Optzr(RidgeFeatureSelection,X,X_train,y_train)
rf  = Swarm_Optzr(RFFeatureSelection,X,X_train,y_train)
hg  = Swarm_Optzr(HGBCFeatureSelection,X,X_train,y_train)
knn = Swarm_Optzr(KNNFeatureSelection,X,X_train,y_train)
svm = Swarm_Optzr(SVMFeatureSelection,X,X_train,y_train)
nb  = Swarm_Optzr(NBFeatureSelection,X,X_train,y_train)



---

---





In [47]:
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values



obj = RidgeFeatureSelection


problem = obj(X_train, y_train)
task = Task(problem, max_iters=100)
algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_features, best_fitness = algorithm.run(task)

selected_features = best_features > 0.8
print('Number of selected features:', selected_features.sum())
print('Selected features:', ', '.join(X.columns[selected_features].tolist()))




model_selected = SVC()
model_all = SVC()

model_selected.fit(X_train[:, selected_features], y_train)
print('Subset accuracy:', model_selected.score(X_test[:, selected_features], y_test))

model_all.fit(X_train, y_train)
print('All Features Accuracy:', model_all.score(X_test, y_test))

Number of selected features: 27
Selected features: nni_min, hr_max, nni_diff_min, rmssd, nn20, pnn20, tinn_n, fft_peak_low, fft_abs_low, fft_log_high, fft_ratio, lomb_peak_medium, lomb_abs_low, lomb_abs_high, lomb_rel_high, lomb_log_low, lomb_norm_high, lomb_ratio, lomb_total, ar_abs_medium, ar_rel_low, ar_log_low, ar_log_medium, sd1, sd2, sd_ratio, sampen
Subset accuracy: 0.6666666666666666
All Features Accuracy: 0.6666666666666666


In [19]:
best_features

array([0.63721682, 0.        , 1.        , 0.        , 0.07668934,
       0.32606324, 1.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.86882527, 0.        , 0.        ,
       0.        , 1.        , 1.        , 1.        , 0.        ,
       0.        , 0.14832107, 1.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       1.        , 0.        , 1.        , 0.76165965, 0.14398781,
       1.        , 1.        , 0.34187019, 0.        , 1.        ,
       0.        , 1.        , 1.        , 0.        , 1.        ,
       0.63431986, 1.        , 0.        , 0.43109561, 1.        ,
       1.        , 0.        , 0.30711802, 0.39366187, 0.        ,
       0.        , 1.        , 1.        , 1.        , 0.        ,
       0.92228781, 0.        , 0.        ])

In [18]:
X.columns[selected_features].tolist()

['nni_counter',
 'nni_min',
 'hr_max',
 'nni_diff_min',
 'rmssd',
 'nn20',
 'pnn20',
 'tinn_n',
 'fft_peak_low',
 'fft_abs_low',
 'fft_log_high',
 'fft_ratio',
 'lomb_peak_medium',
 'lomb_abs_low',
 'lomb_abs_high',
 'lomb_rel_low',
 'lomb_rel_high',
 'lomb_log_low',
 'lomb_norm_high',
 'lomb_ratio',
 'lomb_total',
 'ar_abs_medium',
 'ar_abs_high',
 'ar_rel_low',
 'ar_log_low',
 'ar_log_medium',
 'sd1',
 'sd2',
 'sd_ratio',
 'sampen']